### AutoGen

A framework for building AI agents and applications

https://microsoft.github.io/autogen/stable/index.html

In [1]:
# !pip install -U "autogen-agentchat" "autogen-ext[openai]"
from autogen_agentchat.agents import AssistantAgent
from autogen_ext.models.openai import OpenAIChatCompletionClient
from dotenv import load_dotenv
load_dotenv()

True

In [3]:
agent = AssistantAgent("assistant", OpenAIChatCompletionClient(model="gpt-4o-mini"))
await agent.run(task="Say 'Hello World!'")

TaskResult(messages=[TextMessage(source='user', models_usage=None, content="Say 'Hello World!'", type='TextMessage'), TextMessage(source='assistant', models_usage=RequestUsage(prompt_tokens=41, completion_tokens=3), content='Hello World!', type='TextMessage')], stop_reason=None)

### Quickstart

In [4]:
from autogen_agentchat.agents import AssistantAgent
from autogen_agentchat.teams import RoundRobinGroupChat
from autogen_agentchat.ui import Console
from autogen_ext.models.openai import OpenAIChatCompletionClient


In [5]:
# Define a tool
async def get_weather(city: str) -> str:
    return f"The weather in {city} is 73 degrees and Sunny."

# Define an agent
weather_agent = AssistantAgent(
    name="weather_agent",
    model_client=OpenAIChatCompletionClient(model="gpt-4o-mini"),
    tools=[get_weather],
)

# Define a team with a single agent and maximum auto-gen turns of 1.
agent_team = RoundRobinGroupChat([weather_agent], max_turns=1)

while True:
    # Get user input from the console.
    user_input = input("Enter a message (type 'exit' to leave): ")
    if user_input.strip().lower() == "exit":
        break
    # Run the team and stream messages to the console.
    stream = agent_team.run_stream(task=user_input)
    await Console(stream)

---------- user ----------
hangzhou
---------- weather_agent ----------
[FunctionCall(id='call_T4UV6X7aSCzFA5TSPF4cCB4A', arguments='{"city":"Hangzhou"}', name='get_weather')]
---------- weather_agent ----------
[FunctionExecutionResult(content='The weather in Hangzhou is 73 degrees and Sunny.', call_id='call_T4UV6X7aSCzFA5TSPF4cCB4A')]
---------- weather_agent ----------
The weather in Hangzhou is 73 degrees and Sunny.


### Models

In [6]:
from autogen_ext.models.openai import OpenAIChatCompletionClient
from autogen_core.models import UserMessage

openai_model_client = OpenAIChatCompletionClient(model="gpt-4o-mini")
result = await openai_model_client.create([UserMessage(content="What is the capital of France?", source="user")])

print(result)

finish_reason='unknown' content='The capital of France is Paris.' usage=RequestUsage(prompt_tokens=15, completion_tokens=7) cached=False logprobs=None


### Message

Types of Messages

At a high level, messages in AgentChat can be categorized into two types: agent-agent messages and an agent’s internal events and messages.

In [7]:
from autogen_agentchat.messages import TextMessage

text_message = TextMessage(content="Hello, world!", source="User")

text_message

TextMessage(source='User', models_usage=None, content='Hello, world!', type='TextMessage')

In [42]:
from io import BytesIO

import requests
from autogen_agentchat.messages import MultiModalMessage
from autogen_core import Image as AGImage
from PIL import Image

pil_image = Image.open(BytesIO(requests.get("https://picsum.photos/300/200").content))
img = AGImage(pil_image)
multi_modal_message = MultiModalMessage(content=["Can you describe the content of this image?", img], source="User")

multi_modal_message

MultiModalMessage(source='User', models_usage=None, content=['Can you describe the content of this image?', <autogen_core._image.Image object at 0x1166120d0>], type='MultiModalMessage')

### Agents

In [1]:
from autogen_agentchat.agents import AssistantAgent
from autogen_agentchat.messages import TextMessage
from autogen_agentchat.ui import Console
from autogen_core import CancellationToken
from autogen_ext.models.openai import OpenAIChatCompletionClient


# Define a tool that searches the web for information.
async def web_search(query: str) -> str:
    """Find information on the web"""
    return "AutoGen is a programming framework for building multi-agent applications."


# Create an agent that uses the OpenAI GPT-4o model.
model_client = OpenAIChatCompletionClient(model="gpt-4o-mini")
agent = AssistantAgent(
    name="assistant",
    model_client=model_client,
    tools=[web_search],
    system_message="Use tools to solve tasks.",
)

In [2]:
response = await agent.on_messages(
    [TextMessage(content="Find information on AutoGen", source="user")],
    cancellation_token=CancellationToken(),
)

print("Inner Messages:")
print(response.inner_messages)
print("Chat Message:")
print(response.chat_message)


Inner Messages:
[ToolCallRequestEvent(source='assistant', models_usage=RequestUsage(prompt_tokens=61, completion_tokens=15), content=[FunctionCall(id='call_kQLUCGml2bo0NHqYzF7AwTTJ', arguments='{"query":"AutoGen"}', name='web_search')], type='ToolCallRequestEvent'), ToolCallExecutionEvent(source='assistant', models_usage=None, content=[FunctionExecutionResult(content='AutoGen is a programming framework for building multi-agent applications.', call_id='call_kQLUCGml2bo0NHqYzF7AwTTJ')], type='ToolCallExecutionEvent')]
Chat Message:
source='assistant' models_usage=None content='AutoGen is a programming framework for building multi-agent applications.' type='ToolCallSummaryMessage'


In [3]:
response = await Console(
    agent.on_messages_stream(
        [TextMessage(content="Find information on AutoGen", source="user")],
        cancellation_token=CancellationToken(),
    )
)

---------- assistant ----------
[FunctionCall(id='call_JWGi96gtQlaQMfXSyzBiyRGl', arguments='{"query":"AutoGen framework multi-agent applications"}', name='web_search')]
---------- assistant ----------
[FunctionExecutionResult(content='AutoGen is a programming framework for building multi-agent applications.', call_id='call_JWGi96gtQlaQMfXSyzBiyRGl')]
---------- assistant ----------
AutoGen is a programming framework for building multi-agent applications.


In [4]:
async for response in agent.on_messages_stream(
    [TextMessage(content="Find information on AutoGen", source="user")],
    cancellation_token=CancellationToken(),
):
    print(response)

source='assistant' models_usage=RequestUsage(prompt_tokens=159, completion_tokens=15) content=[FunctionCall(id='call_EKx3hTTosyvbd9AwidzR8fLw', arguments='{"query":"AutoGen"}', name='web_search')] type='ToolCallRequestEvent'
source='assistant' models_usage=None content=[FunctionExecutionResult(content='AutoGen is a programming framework for building multi-agent applications.', call_id='call_EKx3hTTosyvbd9AwidzR8fLw')] type='ToolCallExecutionEvent'
Response(chat_message=ToolCallSummaryMessage(source='assistant', models_usage=None, content='AutoGen is a programming framework for building multi-agent applications.', type='ToolCallSummaryMessage'), inner_messages=[ToolCallRequestEvent(source='assistant', models_usage=RequestUsage(prompt_tokens=159, completion_tokens=15), content=[FunctionCall(id='call_EKx3hTTosyvbd9AwidzR8fLw', arguments='{"query":"AutoGen"}', name='web_search')], type='ToolCallRequestEvent'), ToolCallExecutionEvent(source='assistant', models_usage=None, content=[FunctionE

In [5]:
import pandas as pd
from autogen_ext.tools.langchain import LangChainToolAdapter
from langchain_experimental.tools.python.tool import PythonAstREPLTool

df = pd.read_csv("https://raw.githubusercontent.com/pandas-dev/pandas/main/doc/data/titanic.csv")
tool = LangChainToolAdapter(PythonAstREPLTool(locals={"df": df}))
model_client = OpenAIChatCompletionClient(model="gpt-4o-mini")
agent = AssistantAgent(
    "assistant", tools=[tool], model_client=model_client, 
    system_message=f"Use the `df` variable to access the dataset,column names are {df.columns}.",
    reflect_on_tool_use=True
)

In [ ]:
await Console(
    agent.on_messages_stream(
        [TextMessage(content="What's the average age of the passengers?", source="user")], CancellationToken()
    ),
)

Response(chat_message=TextMessage(source='assistant', models_usage=RequestUsage(prompt_tokens=772, completion_tokens=15), content='The average age of the passengers is approximately 29.7 years.', type='TextMessage'), inner_messages=[])

In [10]:
await Console(
    agent.on_messages_stream(
        [TextMessage(content="What is the average age of survivors? Please display the results for all genders as well as separately for each gender.", source="user")], CancellationToken()
    )
)

Response(chat_message=TextMessage(source='assistant', models_usage=RequestUsage(prompt_tokens=592, completion_tokens=47), content='The average age of all survivors is approximately 28.3 years.\n\nWhen broken down by gender:\n- Average age of female survivors: approximately 28.8 years\n- Average age of male survivors: approximately 27.3 years', type='TextMessage'), inner_messages=[ToolCallRequestEvent(source='assistant', models_usage=RequestUsage(prompt_tokens=543, completion_tokens=81), content=[FunctionCall(id='call_WRiHoJyUg9s33RMcKjYg3G1t', arguments='{"query": "df[df[\'Survived\'] == 1][\'Age\'].mean()"}', name='python_repl_ast'), FunctionCall(id='call_YnEbXIjD57CCC4ZICKekFG01', arguments='{"query": "df[df[\'Survived\'] == 1].groupby(\'Sex\')[\'Age\'].mean()"}', name='python_repl_ast')], type='ToolCallRequestEvent'), ToolCallExecutionEvent(source='assistant', models_usage=None, content=[FunctionExecutionResult(content='28.343689655172415', call_id='call_WRiHoJyUg9s33RMcKjYg3G1t'), 